In [ ]:
# Coluna do tamanho dos genomas isolados em bp (pares de base)

## Primeiro vamos encontrar os arquivos de genoma no diretório atual 
ls GCF_006543135.1_ASM654313v1*

## Depois utilizar este codigo para calcular o tamanho do genoma de uma sequencia em fasta 
awk '/^>/ {next} {total += length($0)} END {print total}' GCF_006543135.1_ASM654313v1_genomic.fna 

awk '/^>/ {next} {total += length($0)} END {print total}' incomplete_genomes.fa

In [ ]:
# Coluna do tamanho dos genomas isolados em mb (Megabyte)
cd /path/
ls -lhs

In [ ]:
# Script para rodar o CD-HIT em arquivos .is.fna dentro de um diretório específico, de forma recursiva e automática.
# Primeiro executamos o nano para criar o script, depois rodamos o script com o comando bash run_cdhit.sh
nano
# Apos isso copiamos e colamos o seguinte código no editor nano:
#################################################################################################################################################################################
#!/bin/bash

# Diretório base
BASE_DIR="/mnt/dados/victor_baca/outputs/isescan"

# Verificar se o diretório existe
if [ ! -d "$BASE_DIR" ]; then
    echo "Erro: Diretório $BASE_DIR não encontrado!"
    exit 1
fi

echo "Processando arquivos .is.fna com CD-HIT..."

# Encontrar todos os arquivos .is.fna e processar
find "$BASE_DIR" -name "*.is.fna" -type f | while read -r fna_file; do
    # Obter o diretório pai do arquivo
    parent_dir=$(dirname "$fna_file")
    
    # Obter o nome do subdiretório
    dirname=$(basename "$parent_dir")
    
    # Nome do arquivo de saída
    output_file="$parent_dir/${dirname}.fa"
    
    echo "Processando: $dirname"
    echo "  Input: $fna_file"
    echo "  Output: $output_file"
    
    # Executar CD-HIT
    export PATH=$PATH::/mnt/dados/victor_baca/tools/cdhit
    cd-hit -i "$fna_file" -o "$output_file" -s 0.9 -g 1
    
    # Verificar se o CD-HIT foi executado com sucesso
    if [ $? -eq 0 ]; then
        echo "  ✓ Sucesso!"
    else
        echo "  ✗ Erro ao processar $fna_file"
    fi
    
    echo ""
done

echo "Processamento concluído!"
echo "Arquivos .fa criados nos respectivos subdiretórios"

#################################################################################################################################################################################
# Ctrl + O : para salvar o arquivo, salvar como -> run_cdhit.sh
# Ctrl + X : para sair do editor nano

# Agora vamos tornar o script executável
chmod +x run_cdhit.sh 

# E finalmente, vamos rodar o script
./run_cdhit.sh

In [ ]:
# Script para a contagem dos ISs 

#!/usr/bin/env python3
"""
Extrator e Contador de Elementos IS (Insertion Sequences) de arquivos FASTA
Autor: Assistente Claude
Uso: python3 is_extractor.py <arquivo_fasta> [arquivo_saida]
"""

import sys
import re
from collections import defaultdict, Counter
import argparse

def parse_fasta(filename):
    """
    Parse um arquivo FASTA e retorna um dicionário com sequências
    """
    sequences = {}
    current_header = None
    current_sequence = []
    
    try:
        with open(filename, 'r') as file:
            for line in file:
                line = line.strip()
                if line.startswith('>'):
                    # Se já temos uma sequência anterior, salva ela
                    if current_header:
                        sequences[current_header] = ''.join(current_sequence)
                    
                    current_header = line[1:]  # Remove o '>'
                    current_sequence = []
                else:
                    if line:  # Ignora linhas vazias
                        current_sequence.append(line)
            
            # Salva a última sequência
            if current_header:
                sequences[current_header] = ''.join(current_sequence)
                
    except FileNotFoundError:
        print(f"Erro: Arquivo '{filename}' não encontrado!")
        sys.exit(1)
    except Exception as e:
        print(f"Erro ao ler arquivo: {e}")
        sys.exit(1)
    
    return sequences

def extract_is_elements(sequences):
    """
    Extrai elementos IS das sequências baseado nos padrões do cabeçalho
    """
    is_elements = {}
    is_types = Counter()
    
    # Padrão para identificar elementos IS no cabeçalho
    # Busca por padrões como "IS21_259", "IS3_176", "ISL3_111", "IS30_241"
    is_pattern = re.compile(r'IS[A-Z]*\d*_\d+')
    
    for header, sequence in sequences.items():
        # Procura por padrões IS no cabeçalho
        is_matches = is_pattern.findall(header)
        
        if is_matches:
            for is_match in is_matches:
                # Extrai o tipo de IS (ex: IS21, IS3, ISL3, IS30)
                is_type_match = re.match(r'(IS[A-Z]*\d*)', is_match)
                if is_type_match:
                    is_type = is_type_match.group(1)
                    is_types[is_type] += 1
                    
                    # Armazena a sequência completa com informações adicionais
                    is_elements[is_match] = {
                        'header': header,
                        'sequence': sequence,
                        'type': is_type,
                        'length': len(sequence)
                    }
    
    return is_elements, is_types

def write_is_sequences(is_elements, output_file):
    """
    Escreve as sequências IS em um arquivo FASTA
    """
    try:
        with open(output_file, 'w') as file:
            for is_id, data in is_elements.items():
                file.write(f">{data['header']}\n")
                # Quebra a sequência em linhas de 80 caracteres
                sequence = data['sequence']
                for i in range(0, len(sequence), 80):
                    file.write(f"{sequence[i:i+80]}\n")
        print(f"Sequências IS salvas em: {output_file}")
    except Exception as e:
        print(f"Erro ao escrever arquivo de saída: {e}")

def print_statistics(is_elements, is_types):
    """
    Imprime estatísticas dos elementos IS encontrados
    """
    print("\n" + "="*60)
    print("RELATÓRIO DE ELEMENTOS IS ENCONTRADOS")
    print("="*60)
    
    print(f"\nTotal de elementos IS encontrados: {len(is_elements)}")
    
    if not is_elements:
        print("Nenhum elemento IS foi encontrado no arquivo.")
        return
    
    print(f"\nTipos de IS encontrados: {len(is_types)}")
    print("\nContagem por tipo de IS:")
    print("-" * 30)
    for is_type, count in sorted(is_types.items()):
        print(f"{is_type:<15}: {count:>5}")
    
    print(f"\nEstatísticas de comprimento:")
    print("-" * 30)
    lengths = [data['length'] for data in is_elements.values()]
    print(f"Comprimento mínimo: {min(lengths):,} bp")
    print(f"Comprimento máximo: {max(lengths):,} bp")
    print(f"Comprimento médio:  {sum(lengths)//len(lengths):,} bp")
    
    print(f"\nDetalhes dos elementos IS:")
    print("-" * 50)
    for is_id, data in sorted(is_elements.items()):
        print(f"{is_id:<15} | {data['type']:<10} | {data['length']:>6,} bp")

def main():
    parser = argparse.ArgumentParser(
        description='Extrai e conta elementos IS de arquivos FASTA',
        formatter_class=argparse.RawDescriptionHelpFormatter,
        epilog="""
Exemplos de uso:
  python3 is_extractor.py Homo_sapiens.fa
  python3 is_extractor.py Homo_sapiens.fa elementos_is.fasta
  python3 is_extractor.py input.fasta output.fasta
        """
    )
    
    parser.add_argument('input_file', help='Arquivo FASTA de entrada')
    parser.add_argument('output_file', nargs='?', 
                       help='Arquivo FASTA de saída (opcional)')
    parser.add_argument('--verbose', '-v', action='store_true',
                       help='Saída mais detalhada')
    
    args = parser.parse_args()
    
    # Define arquivo de saída padrão se não especificado
    if not args.output_file:
        base_name = args.input_file.rsplit('.', 1)[0]
        args.output_file = f"{base_name}_IS_elements.fasta"
    
    print(f"Processando arquivo: {args.input_file}")
    
    # Parse do arquivo FASTA
    sequences = parse_fasta(args.input_file)
    print(f"Total de sequências lidas: {len(sequences)}")
    
    # Extração dos elementos IS
    is_elements, is_types = extract_is_elements(sequences)
    
    # Saída das estatísticas
    print_statistics(is_elements, is_types)

if __name__ == "__main__":
    main()

# Ctrl+O e adicionar o nome do arquivo (is_extractor.py)
# Ctrl+X = para sair

In [ ]:
# Coluna dos TEs e quantidade de ISs

## Utilizando o scritpt
python /mnt/dados/victor_baca/outputs/isescan/is_extractor.py genoma.fa